## Implement by neural network (nn)

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [53]:
df_processed = pd.read_csv('data/train_processed.csv')
X = df_processed.drop('Transported', axis=1).values
y = df_processed['Transported'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
print(type(X_train))
print(X_train.shape[1])

<class 'torch.Tensor'>
13


In [54]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 8)
        self.layer2 = nn.Linear(8, 4)
        self.layer3 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        x = torch.relu(self.layer1(features))
        x = torch.relu(self.layer2(x))
        x = self.layer3(x)
        x = self.sigmoid(x)
        return x

## Loss Function

In [55]:
criterion = nn.BCELoss()
learning_rate = 0.001
model = BinaryClassifier(X.shape[1])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train the model

In [56]:
num_epochs = 2000
for epoch in range(num_epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/2000], Loss: 0.5765
Epoch [200/2000], Loss: 0.5292
Epoch [300/2000], Loss: 0.5146
Epoch [400/2000], Loss: 0.5080
Epoch [500/2000], Loss: 0.4919
Epoch [600/2000], Loss: 0.4859
Epoch [700/2000], Loss: 0.4831
Epoch [800/2000], Loss: 0.4813
Epoch [900/2000], Loss: 0.4792
Epoch [1000/2000], Loss: 0.4770
Epoch [1100/2000], Loss: 0.4744
Epoch [1200/2000], Loss: 0.4706
Epoch [1300/2000], Loss: 0.4658
Epoch [1400/2000], Loss: 0.4607
Epoch [1500/2000], Loss: 0.4550
Epoch [1600/2000], Loss: 0.4512
Epoch [1700/2000], Loss: 0.4474
Epoch [1800/2000], Loss: 0.4450
Epoch [1900/2000], Loss: 0.4432
Epoch [2000/2000], Loss: 0.4419


In [57]:
with torch.no_grad():
    predict = model(X_test)

predict_vals = predict >= 0.5
predict_vals = predict_vals.squeeze().numpy().T
print(y_test[:5])
print(predict_vals[:5])
print(np.sum(predict_vals == y_test))
acc = np.sum(predict_vals == y_test) / len(y_test)
print("Accuracy: ", acc)

[False False  True False  True]
[False False  True  True  True]
1387
Accuracy:  0.7975848188614146
